### L1 Filler Data
This notebook tries to use L0 and L2 data to fill in missing L1 data. This attempt was unsuccessful but keeping the code here for reference.

In [50]:
import pandas as pd
import os
import pickle

In [51]:
with open("/Users/justinbarrett/Desktop/Data_Science/Upwork/mag_model/MAG-model-code/ITER1/iteration_final/basic_word_tokenized_500_test/vocab/topics_vocab.pkl", "rb") as f:
    target_vocab = pickle.load(f)

In [58]:
vocab = pd.DataFrame(zip(target_vocab.keys(), target_vocab.values()), 
                     columns=['normalized_name_2', 'id'])[['normalized_name_2']]

In [8]:
tags = pd.read_parquet("fields_of_study_table.parquet", columns=['field_of_study_id','normalized_name','level']) \
.dropna(subset=['normalized_name'])
tags['level'] = tags['level'].fillna(0).astype('int')
tags.head()

,field_of_study_id,normalized_name,level
0,13818915,2 3 tree,5
1,82747516,widow s peak,2
2,156334865,brianchon s theorem,5
3,2775938904,morphea,2
4,2776538702,philiris,4


In [5]:
children = pd.read_parquet("fields_of_study_children_table.parquet")
children.head()

,field_of_study_id,child_field_of_study_id
0,40700,2775988158
1,40700,2993502798
2,58166,37754750
3,58166,2989026478
4,116537,2987322126


In [33]:
i=0
level = tags[tags['level']==i].copy()
level.columns = [f"field_of_study_id_{i}", f"normalized_name_{i}", f"level_{i}"]
only_level_children_1 = level \
    .merge(children.rename(columns={'field_of_study_id': f'field_of_study_id_{i}'}), 
                                    how='inner', on=f'field_of_study_id_{i}') \
    .merge(tags, how='inner', left_on='child_field_of_study_id', right_on='field_of_study_id') \
    .drop(['child_field_of_study_id'], axis=1)

only_level_children_1.columns = [f"field_of_study_id_{i}", f"normalized_name_{i}", f"level_{i}"] + \
                                [f"field_of_study_id_{i+1}", f"normalized_name_{i+1}", f"level_{i+1}"]

only_level_children_2 = only_level_children_1 \
    .merge(children.rename(columns={'field_of_study_id': f'field_of_study_id_{i+1}'}), 
                                    how='inner', on=f'field_of_study_id_{i+1}') \
    .merge(tags, how='inner', left_on='child_field_of_study_id', right_on='field_of_study_id') \
    .drop(['child_field_of_study_id'], axis=1)

only_level_children_2.columns = [f"field_of_study_id_{i}", f"normalized_name_{i}", f"level_{i}"] + \
                                [f"field_of_study_id_{i+1}", f"normalized_name_{i+1}", f"level_{i+1}"] + \
                                [f"field_of_study_id_{i+2}", f"normalized_name_{i+2}", f"level_{i+2}"]

In [59]:
filler_L1 = only_level_children_2[['normalized_name_0','normalized_name_1','normalized_name_2']] \
.merge(vocab, how='inner', on='normalized_name_2').copy()
filler_L1.shape

(74564, 3)

In [60]:
filler_L1['normalized_name_2'].nunique()

27508

In [61]:
filler_L1.drop_duplicates(subset=['normalized_name_0','normalized_name_2']).shape

(50826, 3)

In [70]:
test_filler = filler_L1.groupby(['normalized_name_0','normalized_name_2']).count().sort_values('normalized_name_1', ascending=False)

In [71]:
test_filler.shape

(50826, 1)

In [80]:
one_or_two = test_filler[test_filler['normalized_name_1']<3] \
.reset_index()[['normalized_name_0','normalized_name_2']].copy()

In [84]:
one_or_two.shape

(45511, 2)

In [87]:
final_filler = filler_L1.drop_duplicates(subset=['normalized_name_0','normalized_name_2'], keep=False)
final_filler.shape

(32630, 3)

In [88]:
one_two_filler = filler_L1.merge(one_or_two, how='inner', on=['normalized_name_0','normalized_name_2'])
one_two_filler.shape

(58392, 3)

In [89]:
final_filler.to_parquet("single_instance_filler.parquet")

In [90]:
one_two_filler.to_parquet("one_two_instance_filler.parquet")